# Three states MIS algorithm

After simulating a guess algorithm for a 2 level rydberg atom system trying to solve MIS with adiabatic quantum computing

In [58]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
# The library for quantum adiabatic simulations.
from qutip import * 
from scipy import *
# The library used for the graph representation.  
import networkx as nx
from guess_algorithm_helper import *
from random import randint
import ipynb.fs.defs.guess_algorithm as algo

and follow up complexity testing compared to classical algorithms

In [59]:
import ipynb.fs.defs.guess_algorithm_tester as tester

We got mixed results. When the guess was close to the expected results, the guess did amazing and we saw huge improvement. 
However, when the guess wasn't as close, as expected the normal algorithm which starts with a ground state for all atoms did better. 

Hence, we want to implement Luby's algorithm, which uses degree based guesses and is one of the best MIS solvers in classical algorithms. Meaning that our goal is implementing something like:
$$|\psi_0 \rangle=\sqrt{\frac{1}{dv}} |r \rangle + \sqrt{1-\frac{1}{dv}} |g \rangle$$

Which would be great, but is also impossible. That is because this will have a probability of neighboring vertexs being in Rydberg states.
To solve that, we will attempt to create a three level system. In the beginning we will only have states at $|0 \rangle$ and $|1 \rangle$. where $|1 \rangle$ is choosing the vertex and $|0 \rangle$ is not choosing the vertex. Meaning that 
$$|\psi_0 \rangle=\sqrt{\frac{1}{dv}} |1 \rangle + \sqrt{1-\frac{1}{dv}} |0 \rangle$$
 
Just like we wanted. And the starting hamiltonain will also reflect that. After some time, using some rabi frequency $\Omega(t)$, we will move as many atoms as possible from $|1 \rangle$ to $|r \rangle$. Which can be looked as as the original 2 states algorithm with $|1 \rangle$ being the ground. 

Since at the start the rydberg state probability is 0, all initial states will be viable ground states without any need for $H_0$ to be implemented at the start of the run.Hence we get:

$$\begin{cases}
H_0 = \sum_{n=1}^NI - \sigma_{x_{1,r}} ^n \\
H_p = -\Delta n_v \\
H_{int} = \sum_{v<w}V/x^6(|\vec{x}_v-\vec{x}_w|)n_vn_w
\end{cases}
$$

With 
$$\sigma_{x_{1,r}} = \begin{pmatrix}
0 & 0 & 0 \\
0 & 0 & 1 \\
0 & 1 & 0
\end{pmatrix}$$

Using that we can build a full hamiltonian:
$$H_{tot} = H_0 \Omega(t) + H_p\Delta(t) + H_{int}$$

## Constructing operators 
We need speciel operators because we now have a unique $\sigma_x$ to convert the atom from $|1\rangle $ to $|r\rangle$

In [60]:
from qutip import qeye, basis, Qobj, tensor, sigmaz

def generate_operators(N, num_states=3):
    """
    Generates useful quantum operators for a system of N states, including a custom sigma_x operator.

    Parameters:
    N (int): The number of operators to generate.
    num_states (int): The dimension of the Hilbert space for each state (e.g., 2 for qubits, 3 for qutrits).

    Returns:
    tuple: A tuple containing lists of tensor products of generalized operators.
           (sx_list, sz_list, one_list, I_N)
    """
    # Identity operator for the specified number of states
    si = qeye(num_states)

    # Custom σ_x operator for states |1> and |2>, converted to Qobj
    sx_custom = Qobj([[0, 0, 0],
                      [0, 0, 1],
                      [0, 1, 0]])

    # Initialize empty lists for storing operators
    sx_list = []
    sz_list = []
    one_list = []

    # Identity tensor for N states
    I_N = tensor([si] * N)

    # Generate the operators
    for n in range(N):
        # Initialize a list of identities for the tensor product
        op_list = [si] * N

        # Replace the n-th position with the custom sx operator
        op_list[n] = sx_custom
        sx_list.append(tensor(op_list))

        # Replace the n-th position with the sz operator
        sz = Qobj(sigmaz().full()[:num_states, :num_states])  # Define sz for the 3-state system
        op_list[n] = sz
        sz_list.append(tensor(op_list))

        # Replace the n-th position with |1><1|
        one = basis(num_states, 1)  # |1><1| in higher dimensions
        op_list[n] = one * one.dag()
        one_list.append(tensor(op_list))

    return sx_list, sz_list, one_list, I_N

## Hamiltonian building

In [61]:
def default_guess_function(graph, node_index, degree_min):
    """Default guess function based on node degree."""
    if len(graph.degree)==2:
        return node_index==0
    return 1 / max(1, graph.degree[node_index]) >= degree_min

def initialize_states_and_hamiltonians(params, graph_instance, guess_func = default_guess_function, use_position=False):
    """
    Initializes the Hamiltonians and state vectors based on the graph properties.
    
    Parameters:
    params (dict): A dictionary of system parameters including DEGREE_MIN and delta.
    graph_instance (BaseGraph): An instance of the graph class (e.g., ChainGraph).
    guess_func (function): A function that generates a guess based on the node's properties.
    use_position (bool): Whether to use the graph's positional data for blockade radius calculation.

    Returns:
    tuple: (hamiltonians, psi_states)
    """
    N = len(graph_instance.graph.nodes)  # Number of nodes in the graph
    sx_list, _, one_list, I_N = generate_operators(N)

    # Classical Hamiltonian
    CL_H0 = I_N
    for n in range(N):
        CL_H0 -= sx_list[n]

    # Guess Hamiltonian
    G = graph_instance.graph  # Ensure G is defined from the graph instance

    # State preparation using the precomputed guesses
    guess_psi_list = [(np.sqrt(1/G.degree[n]))*basis(3, 1) +  (np.sqrt(1-1/G.degree[n]))*basis(3, 0) for n in range(N)]
    guess_psi0 = tensor(guess_psi_list)

    Hp = 0
    H_int = 0
    for v in range(N):
        Hp -= one_list[v] 
        for w in range(v+1, N):
            if use_position:
                dist_x = G.nodes[v]['pos'][0] - G.nodes[w]['pos'][0]
                dist_y = G.nodes[v]['pos'][1] - G.nodes[w]['pos'][1]
                dist = np.sqrt(dist_x**2 + dist_y**2)
            else:
                dist = params["blockade_radius"]*params['vertex_placement'] if (v, w) in G.edges else params["blockade_radius"] * 3*(1/params['vertex_placement'])
            H_int += params["INTERACTION_STRENGTH"] / ((dist)**6) * one_list[v] * one_list[w]
    
    # Store Hamiltonians in a dictionary
    hamiltonians = {
        "luby": [[CL_H0 , params['Omega']], [Hp, params['Delta']], H_int],
    }

    # Store state vectors in a dictionary
    psi_states = {
        "luby": guess_psi0,
    }
    
    return hamiltonians, psi_states

def solve_hamiltonians(hamiltonians, psi_states, params):
    """
    Transforms Hamiltonians to QobjEvo and solves them using the Schrödinger equation.

    Parameters:
    hamiltonians (dict): A dictionary containing Hamiltonians as arrays.
                         Example: hamiltonians["classic"], hamiltonians["guess"], hamiltonians["setup"]
    psi_states (dict): A dictionary containing initial state vectors.
                       Example: psi_states["classical"], psi_states["guess"]
    params (dict): A dictionary of parameters, including 'taulist'.

    Returns:
    tuple: Two dictionaries - one for Hamiltonians and one for results from solving the Hamiltonians.
    """
    
    # Extract parameters
    taulist = params['taulist']

    # Transform Hamiltonians to QobjEvo
    luby_h_t = QobjEvo(hamiltonians["luby"])
    luby_result = sesolve(luby_h_t, psi_states["luby"], taulist, [], {})

    # Return Hamiltonians and results in separate dictionaries
    hamiltonians_out = {
        "luby": luby_h_t
    }

    results_out = {
        "luby": luby_result
    }

    return hamiltonians_out, results_out

init_params = algo.initialize_system_parameters(vertex_placement = 0.8)
chain_graph = algo.ChainGraph(5)
chain_graph.create_graph()
hamiltonians, psi_states = initialize_states_and_hamiltonians(init_params, chain_graph)
hamiltonians_out, results_out = solve_hamiltonians(hamiltonians, psi_states, init_params)
algo.plot_probabilities(results_out, psi_states, 5, graph_type="Chain", graph=chain_graph, threshold=0.01)

IndexError: list index out of range